# Import data

We will use [Brightway](https://brightway.dev/); however, Brightway automatically corrects some unreasonable uncertainty values, so we need to make sure these corrections are not applied. We do keep our corrections for impossible values (i.e. min == max).

In [1]:
from bw2data import *
from bw2io import *
projects.set_current("OASES uncertainty review")

In [7]:
bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/03/2020 09:01:39
  Finished: 07/03/2020 09:01:40
  Total time elapsed: 00:00:00
  CPU %: 64.10
  Memory %: 1.10
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.92 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations



In [2]:
from bw2io.extractors.ecospold2 import *

In [3]:
class DumbEcospold2DataExtractor(Ecospold2DataExtractor):
    @classmethod
    def extract_uncertainty_dict(cls, obj):
        data = {
            'amount': float(obj.get('amount')),
        }
        if obj.get('formula'):
            data['formula'] = obj.get('formula')

        if hasattr(obj, "uncertainty"):
            unc = obj.uncertainty
            if hasattr(unc, "pedigreeMatrix"):
                data['pedigree'] = dict([(
                    PM_MAPPING[key], int(unc.pedigreeMatrix.get(key)))
                    for key in PM_MAPPING
                ])

            if hasattr(unc, "lognormal"):
                data.update({
                    'uncertainty type': LognormalUncertainty.id,
                    "loc": float(unc.lognormal.get('mu')),
                    "scale": math.sqrt(float(unc.lognormal.get("varianceWithPedigreeUncertainty"))),
                })
                if unc.lognormal.get('variance'):
                    data["scale without pedigree"] = math.sqrt(float(unc.lognormal.get('variance')))
                if data["scale"] <= 0:
                    cls.abort_exchange(data, TOO_LOW.format(data['scale']))
                # This is the only change versus the base class
                #                 elif data["scale"] > 25:
                #                     cls.abort_exchange(data, TOO_HIGH.format(data['scale']))
            elif hasattr(unc, 'normal'):
                data.update({
                    "uncertainty type": NormalUncertainty.id,
                    "loc": float(unc.normal.get('meanValue')),
                    "scale": math.sqrt(float(unc.normal.get('varianceWithPedigreeUncertainty'))),
                })
                if unc.normal.get('variance'):
                    data["scale without pedigree"] = math.sqrt(float(unc.normal.get('variance')))
                if data["scale"] <= 0:
                    cls.abort_exchange(data)
            elif hasattr(unc, 'triangular'):
                data.update({
                    'uncertainty type': TriangularUncertainty.id,
                    'minimum': float(unc.triangular.get('minValue')),
                    'loc': float(unc.triangular.get('mostLikelyValue')),
                    'maximum': float(unc.triangular.get('maxValue'))
                })
                if data["minimum"] >= data["maximum"]:
                    cls.abort_exchange(data)
            elif hasattr(unc, 'uniform'):
                data.update({
                    "uncertainty type": UniformUncertainty.id,
                    "loc": data['amount'],
                    'minimum': float(unc.uniform.get('minValue')),
                    'maximum': float(unc.uniform.get('maxValue')),
                })
                if data["minimum"] >= data["maximum"]:
                    cls.abort_exchange(data)
            elif hasattr(unc, 'undefined'):
                data.update({
                    "uncertainty type": UndefinedUncertainty.id,
                    "loc": data['amount'],
                })
            else:
                raise ValueError("Unknown uncertainty type")
        else:
            data.update({
                "uncertainty type": UndefinedUncertainty.id,
                "loc": data['amount'],
            })
        return data    

In [4]:
from bw2io.importers.ecospold2 import *

In [9]:
class DumbSingleOutputEcospold2Importer(SingleOutputEcospold2Importer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.strategies = [
            normalize_units,
            update_ecoinvent_locations,
            remove_zero_amount_coproducts,
            remove_zero_amount_inputs_with_no_activity,
            remove_unnamed_parameters,
            es2_assign_only_product_with_amount_as_reference_product,
            assign_single_product_as_activity,
            create_composite_code,
            drop_unspecified_subcategories,
            fix_ecoinvent_flows_pre35,
            drop_temporary_outdated_biosphere_flows,
            link_biosphere_by_flow_uuid,
            link_internal_technosphere_by_composite_code,
            delete_exchanges_missing_activity,
            delete_ghost_exchanges,
            # Don't do these corrections to get "true" values
            # remove_uncertainty_from_negative_loss_exchanges,
            # fix_unreasonably_high_lognormal_uncertainties,
            set_lognormal_loc_value,
            convert_activity_parameters_to_list,
            add_cpc_classification_from_single_reference_product,
        ]

In [10]:
dirpath = "/Users/cmutel/Documents/LCA/Ecoinvent/3.6/cutoff/datasets"

In [11]:
ei = DumbSingleOutputEcospold2Importer(
    dirpath=dirpath,
    db_name="ecoinvent 3.6 cutoff",
    extractor=DumbEcospold2DataExtractor
)
ei.apply_strategies()
ei.statistics()

Extracting XML data from 18121 datasets
Extracted 18121 datasets in 94.93 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
Applying strategy: add_cpc_classification_from_single

(18121, 615644, 0)

In [12]:
ei.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:02


Title: Writing activities to SQLite3 database:
  Started: 07/03/2020 09:14:23
  Finished: 07/03/2020 09:15:25
  Total time elapsed: 00:01:02
  CPU %: 83.00
  Memory %: 9.91
Created database: ecoinvent 3.6 cutoff


Brightway2 SQLiteBackend: ecoinvent 3.6 cutoff